--- 
# Sandbox (Faune)
Tests for each part of this project

---

### Imports

In [ ]:
#external libraries
import polars as pl
from dask.distributed import Client, as_completed

#internal libraries
from prompts import run_async_query

## Importing data test

In [20]:
titles = pl.read_csv(source=r"data\cryptopanic_news.csv").select(pl.col("id").alias('newsId'), "title", "newsDatetime")
currency = pl.read_csv(r"data\currency.csv").rename({'id':'currencyId'})
ids = pl.read_csv(r"data\news__currency.csv")

In [46]:
test_list = (
    titles
    .join(ids, on="newsId")
    .join(currency, on='currencyId')
    .filter(pl.col("code").eq('BTC'))
    .with_columns(
        pl.col('newsDatetime').str.to_datetime().cast(pl.Date).alias('date')
    )
    .sort('newsDatetime')
    .select('date','title')
    .with_columns(
        prompts = pl.lit("""
Here is a piece of news:
"%s"
Do you think this news will increase or decrease %s?
Write your answer as:
{increase/decrease/uncertain}:
{confidence (0-1)}:
{magnitude of increase/decrease (0-1)}:
{explanation (less than 25 words)}
        """)
    )
    .with_columns(pl.col('prompts').str.replace(r'%s', pl.col('title')))
    .with_columns(pl.col('prompts').str.replace(r'%s', "BTC"))
).select('prompts').to_series().to_list()[:3]

## Prompting test

### Basic sentences
Adapted from the mistral website

In [ ]:
with Client() as client:
    prompts = test_list

    # submit
    futures = [client.submit(run_async_query, prompt) for prompt in prompts]

    # display
    for future in as_completed(futures):
        result = future.result()
        print("\n--- Result ---\n", result) # store somewhere instead 


--- Result ---
 {increase}: {confidence (0.3)}: {magnitude of increase (0.1)}: This could potentially lead to a minor increase in BTC as a hard fork in Bitcoin Cash might encourage investors to hold more BTC as a safer alternative.

This response is based on general market behavior and should not be taken as financial advice. Always do your own research before making investment decisions.

--- Result ---
 {increase: uncertain}
{confidence (0-1): 0.3}
{magnitude of increase/decrease (0-1): 0.1}
The mention of Zcash as an alternative to Bitcoin might not have a significant impact on Bitcoin's price, as it is not directly comparing Bitcoin to itself. However, if it sparks interest in privacy-focused cryptocurrencies, there might be a minor positive effect on Bitcoin due to increased overall market interest.

--- Result ---
 {increase}: {0.3}
{magnitude of increase (0-1)}: {0.1}
The forking of Bitcoin Cash might divert mining resources away from Bitcoin, potentially leading to a slightly 